# Capstone Workbook 4: Initial Modelling

In [2]:
# Import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Import data 
airbnb_ldn = pd.read_csv('airbnb_ldn_pp.csv')

In [5]:
# drop 'Unnamed: 0'
airbnb_ldn = airbnb_ldn.drop(columns = 'Unnamed: 0')

In [6]:
# View data:
airbnb_ldn.head().T

,0,1,2,3,4
Listing Title,Cozy 2BR house with a garden view,GuestReady - Amazing home with a private garden,Cosy cottage on Richmond Park,"Entire Flat. Free parking, Garden , Richmond park",Maisonette inbetween Richmond Park and Wimbledon
Property Type,Entire home,Entire home,Entire home,Entire rental unit,Private room in rental unit
City,Greater London,Greater London,Greater London,Greater London,Greater London
Zipcode,SW15 3,SW15 3,SW15 3,SW15 3,SW15 3
Number of Reviews,9,11,1,20,0
Bedrooms,2.0,2.0,1.0,2.0,1.0
Bathrooms,2,1,2,1,1
Max Guests,6,4,3,4,2
Airbnb Superhost,0,1,0,0,0
Cleaning Fee (Native),154.8,0.0,0.0,34.8,0.0


The data has now been cleaned, had some initial EDA completed and been preprocessed. 

Some initial models will now be built, starting with a regression model with a L1 penalty. This will help identify which columns are influencial in predicting the target column.

First, the dataframe will be split into the independent and target variables, using just numerical variables for now:

In [16]:
X = airbnb_ldn.select_dtypes(exclude='object').drop(columns = 'Annual Revenue LTM (Native)')
y = airbnb_ldn['Annual Revenue LTM (Native)']

The dependent and target variables will now be split into into train and test sets:

In [17]:
# download required sklearn packages:
from sklearn.model_selection import train_test_split

# splitting data into train and test set:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

As random samples have been taken, the indexes for the dataframes will be reset:

In [18]:
# reset index for X training set:
X_train.reset_index(inplace=True)

# drop created 'index' column:
X_train.drop(columns='index', inplace=True)

C:\Users\blcm9\AppData\Local\Temp\ipykernel_15032\3935498988.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns='index', inplace=True)


In [20]:
# reset index for X_test:
X_test.reset_index(inplace=True)

# drop created 'index column:
X_test.drop(columns='index', inplace=True)

C:\Users\blcm9\AppData\Local\Temp\ipykernel_15032\3863809310.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns='index', inplace=True)


The index for the target column must also be reset:

In [37]:
# complete the same tranformation for the y_test data:
# convert the series to a dataframe:
y_train = y_train.to_frame()

# reset index
y_train.reset_index(inplace=True)

# drop 'index' column
y_train.drop(columns = 'index', inplace=True)

# return column to series
y_train = y_test.squeeze()

Complete the same tranformation for the y_test data:

In [43]:
# convert the series to a dataframe:
y_test = y_test.to_frame()

# reset index
y_test.reset_index(inplace=True)

# drop 'index' column
y_test.drop(columns = 'index', inplace=True)

# return column to series
y_test = y_test.squeeze()

Now the train and test datasets have been transferred, the first model can be made.
A standard scaler will be used and a L1 (lasso) penalty.

In [55]:
from scipy import stats

In [56]:
# import required libraries
import statsmodels.api as sm


ModuleNotFoundError: No module named 'statsmodels'